In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

# batch_input_file = client.files.create(
#   file=open("D:\\Projects(D)\\Fine-Tuned-GPT-2-with-articles-ground-truth\\code\\llamaIndex\\.cache\\batchinput.jsonl", "rb"),
#   purpose="batch"
# )

# batch_input_file_id = batch_input_file.id

# batch = client.batches.create(
#     input_file_id=batch_input_file_id,
#     endpoint="/v1/chat/completions",
#     completion_window="24h",
#     metadata={
#       "description": "nightly eval job"
#     }
# )

In [ ]:
batch.__dict__

In [ ]:
client.batches.retrieve('batch_5TFUPEN2etKNpSWnq4Kh5fi4').__dict__

In [ ]:
content = client.files.content('file-3DR1YeMpP7JHGpHWDwXzsh17')

In [ ]:
print(content.text)

In [ ]:
with open('output_file.jsonl', 'w') as output_file:
    output_file.write(content.text)

In [ ]:
import json
content = json.loads(content.text)

### Simulate extraction

In [ ]:
from database import Database

d = Database(config_dir_path='./code/llamaIndex')
nodes = d.create_or_update_indexes()

In [ ]:
import os
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
root_path = '../..'
cache_dir = os.path.abspath(os.path.join(root_path, './code/llamaIndex/.cache'))

In [ ]:
system_prompt = """\
You are a highly knowledgeable reasearch assistant tasked with generating insightful questions, detailed answers, and \
thorough reasoning based on the provided parts of papers.\
"""

prompt_template_openai="""\
Here is the context:
{context_str}

Using this context, generate 5 specific questions that this context can uniquely answer. Ensure that these questions:
1. Are directly related to the provided context.
2. Highlight unique information or insights from the context.
3. Cannot be easily answered by general knowledge.
----------------------------------------------------------------------------------
<Pair number, representing which QAR you are at, like 1, 2, 3>
Question:<Question content, you should place a specific question which is unlikely to be found elsewhere and is unique comparing with other questions>

Answer:<Answer content, you should place a specific answer combining with the offered context>

Reason:<Reason content, you should explain why this question and answer are unlikely to be found elsewhere and are unique comparing with each other>
----------------------------------------------------------------------------------
Higher-level summaries of surrounding context may be provided \
as well. Try using these summaries to generate better questions \
that this context can answer.
"""

In [ ]:
def create_batch(now, file_path):
    batch_input_file = client.files.create(
        file=open(file_path, "rb"),
        purpose="batch"
    )

    batch_input_file_id = batch_input_file.id

    batch = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )
    batch_path = os.path.join(cache_dir, f"{now}-batchinput-response.json")
    with open(batch_path, 'w') as file:
        json.dump(batch.to_dict(), file, indent=4)

    return batch

In [ ]:
from datetime import datetime

batches = []
node_id = 0
for node in nodes[:2]:
    if node_id % 50000 == 0:
        now = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
        file_path = os.path.join(cache_dir, f"{now}-batchinput.jsonl")
        file = open(file_path, 'w+')
    entry = {
            "custom_id": node.id_,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o",
                "messages": [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompt_template_openai.format(context_str=node.text)}
                    ],
                "max_tokens": 4096
            }
        }
    json_line = json.dumps(entry)
    file.write(json_line + "\n")
    if (node_id+1) % 50000 == 0:
        file.close()
        batch = create_batch(now, file_path)
        batches.append(batch)
    node_id += 1
if (node_id+1) % 50000 != 0:
    file.close()
    batch = create_batch(now, file_path)
    batches.append(batch)

In [ ]:
batch_input_file = client.files.create(
    file=open(file_path, "rb"),
    purpose="batch"
)

In [ ]:
batch_input_file_id = batch_input_file.id

batch = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)
batch_path = os.path.join(cache_dir, f"{now}-batchinput-response.json")


In [ ]:
batches.append(batch)

In [ ]:
batches = [batches[1]]

In [ ]:
batches

In [ ]:
finished_contents = set()
while len(finished_contents) < len(batches):
    # Refresh the batch response
    for batch in batches:
        batch = client.batches.retrieve(batch.id)
        if batch.status == 'completed':
            if batch.request_counts.completed == batch.request_counts.total:
                content = client.files.content(batch.output_file_id)
                finished_contents.add(content)
            else:
                print(f"Get {batch.status} at batch {batch.id}")
                exit()
        if batch.status in ['expired', 'failed', 'cancelled']:
            print(f"Get {batch.status} at batch {batch.id}")
            exit()

In [ ]:
batch = client.batches.retrieve(batch.id)

In [ ]:
batch.status

In [ ]:
batch.to_dict()

In [ ]:
batch.request_counts.completed

In [ ]:
client.files.content(batch.error_file_id).text

In [ ]:
data =[json.loads(response) for response in content.text.strip().split('\n')]
with open('output_file.json', 'w') as output_file:
    json.dump(data, output_file, indent=4)


In [ ]:
data

In [ ]:
for request in data:
    print(request['response']['body']['choices'][0]['message']['content'])

In [ ]:
node_dict = {}
for node in nodes[:2]:
    print(node.id_)
    node_dict[node.id_] = node
    print(node)

In [ ]:
for request in data:
    node_dict[request['custom_id']].metadata["questions_this_excerpt_can_answer_and_corresponding_answers_and_reasons"] = request['response']['body']['choices'][0]['message']['content']

In [ ]:
for node in nodes:
    print(node.get_content(metadata_mode='embed'))

In [172]:
import time
from tqdm import tqdm
import random
uncompleted_batches = [random.randint(1, 100) for _ in range(10)]
total_batches = len(uncompleted_batches)

with tqdm(total=total_batches, desc="Processing batches", unit="batch") as pbar:
    while len(uncompleted_batches) > 0:
        for _ in range(2):  # 10-minute countdown
            # Simulate batch completion
            uncompleted_batches.pop()

        # Update the progress bar
        pbar.n = total_batches - len(uncompleted_batches)
        pbar.refresh()
        time.sleep(2)

Processing batches: 100%|██████████| 10/10 [00:10<00:00,  1.00s/batch]


In [174]:
total_batches = int(len(nodes)/45000) + (len(nodes) % 45000 > 0)

In [179]:
extractor_index_config = {
    "gpt-4o-QAExtractor-batch-interrupted": {
        "need_interrupt": True,
        "force_extract": True
    }
}
extractor_name, extractor_config = next(iter(extractor_index_config.items()))

print(extractor_name)
print(extractor_config)

ValueError: not enough values to unpack (expected 2, got 1)